#### Given the high dimensionality of oour dataset, we want to reduce the number of columns by selecting a subset.

The dimensionality will be further reduced once we aggregate the currently hourly data to dail granularity.

In [1]:
import pandas as pd

In [11]:
weather_data = pd.read_csv("./WeatherFilteredAndLocaltime_hourly.csv")

In [4]:
weather_data.head()

,Unnamed: 0,datetime,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2325840,2020-01-01 01:00:00,-0.2,88.0,-2.0,-3.7,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2325841,2020-01-01 02:00:00,-0.9,88.0,-2.7,-4.4,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2325842,2020-01-01 03:00:00,-1.3,88.0,-3.0,-4.7,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2325843,2020-01-01 04:00:00,-1.0,89.0,-2.6,-4.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2325844,2020-01-01 05:00:00,-1.0,90.0,-2.4,-4.8,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
weather_data.columns

Index(['Unnamed: 0', 'datetime', 'temperature_2m', 'relative_humidity_2m',
       'dew_point_2m', 'apparent_temperature', 'precipitation', 'rain',
       'snowfall', 'snow_depth', 'weather_code', 'pressure_msl',
       'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid',
       'cloud_cover_high', 'et0_fao_evapotranspiration',
       'vapour_pressure_deficit', 'wind_speed_10m', 'wind_speed_100m',
       'wind_direction_10m', 'wind_direction_100m', 'wind_gusts_10m',
       'soil_temperature_0_to_7cm', 'soil_temperature_7_to_28cm',
       'soil_temperature_28_to_100cm', 'soil_temperature_100_to_255cm',
       'soil_moisture_0_to_7cm', 'soil_moisture_7_to_28cm',
       'soil_moisture_28_to_100cm', 'soil_moisture_100_to_255cm',
       'shortwave_radiation', 'direct_radiation', 'diffuse_radiation',
       'direct_normal_irradiance', 'global_tilted_irradiance',
       'terrestrial_radiation', 'shortwave_radiation_instant',
       'direct_radiation_instant', 'diffuse_radia

After an analysis of the way the values were recorded and what they represented, we decided to discard the following columns:

- 'snowfall' : since by keeping 'rain' and 'precipitation' fields it can be deduced (precipitation = rain + snowfall).
- 'snow_depth' : As it's interaction with air quality was deemed less relevant.
- 'cloud_cover_low', 'cloud_cover_mid', and 'cloud_cover_high' : as 'cloud_cover' represents total cover, while others only at certain altitude.
- 'soil_temperature_28_to_100cm', 'soil_temperature_100_to_255cm','soil_moisture_28_to_100cm', 'soil_moisture_100_to_255cm' : to focus on moisture closer to surface
-'shortwave_radiation_instant','direct_radiation_instant', 'diffuse_radiation_instant','direct_normal_irradiance_instant','global_tilted_irradiance_instant','terrestrial_radiation_instant' : as their non-instant counterparts account for the average value for the preceding hour, so we considered it more informative when evaluating dimensionality reduction tradeoff. 

In [12]:
#Therefore
weather_data = weather_data.drop(columns=['Unnamed: 0','snowfall','snow_depth','cloud_cover_low', 'cloud_cover_mid','cloud_cover_high',
    'soil_temperature_28_to_100cm', 'soil_temperature_100_to_255cm','soil_moisture_28_to_100cm', 'soil_moisture_100_to_255cm',
    'shortwave_radiation_instant','direct_radiation_instant', 'diffuse_radiation_instant','direct_normal_irradiance_instant',
    'global_tilted_irradiance_instant','terrestrial_radiation_instant'])

In [13]:
display(weather_data)

,datetime,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,weather_code,pressure_msl,surface_pressure,...,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation
0,2020-01-01 01:00:00,-0.2,88.0,-2.0,-3.7,0.0,0.0,0.0,1030.6,1012.1,...,0.6,4.4,0.364,0.371,0.0,0.0,0.0,0.0,0.0,0.0
1,2020-01-01 02:00:00,-0.9,88.0,-2.7,-4.4,0.0,0.0,0.0,1030.4,1011.9,...,0.6,4.2,0.364,0.371,0.0,0.0,0.0,0.0,0.0,0.0
2,2020-01-01 03:00:00,-1.3,88.0,-3.0,-4.7,0.0,0.0,0.0,1030.2,1011.6,...,0.6,4.1,0.364,0.371,0.0,0.0,0.0,0.0,0.0,0.0
3,2020-01-01 04:00:00,-1.0,89.0,-2.6,-4.8,0.0,0.0,0.0,1029.9,1011.4,...,0.6,3.9,0.364,0.371,0.0,0.0,0.0,0.0,0.0,0.0
4,2020-01-01 05:00:00,-1.0,90.0,-2.4,-4.8,0.0,0.0,0.0,1029.7,1011.2,...,0.6,3.8,0.364,0.371,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35058,2023-12-31 19:00:00,6.9,98.0,6.5,5.0,0.1,0.1,51.0,1011.1,993.4,...,7.3,8.0,0.385,0.353,0.0,0.0,0.0,0.0,0.0,0.0
35059,2023-12-31 20:00:00,6.8,97.0,6.4,5.8,0.2,0.2,51.0,1011.2,993.5,...,7.3,8.0,0.385,0.354,0.0,0.0,0.0,0.0,0.0,0.0
35060,2023-12-31 21:00:00,6.9,97.0,6.4,4.6,1.0,1.0,55.0,1011.3,993.6,...,7.3,8.0,0.396,0.354,0.0,0.0,0.0,0.0,0.0,0.0
35061,2023-12-31 22:00:00,6.9,96.0,6.3,5.6,2.1,2.1,61.0,1010.9,993.2,...,7.2,7.9,0.417,0.357,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
#weather_data.to_csv("hourlyWeather_final.csv")